# MintPy Processing

This notebook performs the MintPy timeseries processing. If you you're processing an ascending and descending track you'll need to run this workflow twice. Once for each track. I've had to add some strange working directory management to get this workflow to run on my local machine (run the load_data step in the directory the hyp3 data is located in, then copy it into a processing directory) but feel free to change this workflow if your machine is different.

## setup

In [ ]:
%matplotlib inline
import glob
import sys
import shutil
import os
import matplotlib.pyplot as plt
from mintpy.utils import readfile, writefile, utils as ut
from mintpy import workflow, view, tsview, plot_network, plot_transection, plot_coherence_matrix

from mintpy.objects import timeseries
from datetime import datetime
import numpy as np
from skimage.morphology import remove_small_objects

from scipy.stats import linregress
import pandas as pd
import matplotlib
import datetime

['load_data', 'modify_network', 'reference_point', 'quick_overview', 'correct_unwrap_error']
  ['invert_network', 'correct_LOD', 'correct_SET', 'correct_troposphere', 'deramp', 'correct_topography']
  ['residual_RMS', 'reference_date', 'velocity', 'geocode', 'google_earth', 'hdfeos5']

In [ ]:
cd /home/micromamba/data

In [ ]:
cp /home/micromamba/scripts/configs/desc_upper.cfg ./hyp3.cfg

## load_data

In [ ]:
!load_data.py -t hyp3.cfg

## double_difference

This the only step that isn't included in the normal MintPy `smallbaselineApp` workflow. It highlights local deformation, but obscures the presence of large-scale (greater than 10 km scale) deformation and unwrapping errors. See [Bekaert et. al 2020](https://www.sciencedirect.com/science/article/pii/S0034425720303539) for more details.

In [ ]:
desired_width = 2000
pixel_size = 40

x = 750
n_pixels = np.floor((desired_width / 40) /2)
width = ((n_pixels*2)+1)*400
print(f'N pixels: {n_pixels}\nActual width: {width}')

In [ ]:
from skimage import morphology
filter_par = [2,25]

local_kernel = morphology.disk(filter_par[0], np.float32)
regional_kernel = morphology.disk(filter_par[1], np.float32)

center = slice(filter_par[1]-filter_par[0],filter_par[1]+filter_par[0]+1)
print(center,center)
regional_kernel[center,center] =  regional_kernel[center,center] - local_kernel

local_kernel = local_kernel / local_kernel.flatten().sum()
# local_filt = ndimage.convolve(data, local_kernel)

regional_kernel = regional_kernel / regional_kernel.flatten().sum()
# regional_filt = ndimage.convolve(data, regional_kernel)

f, (ax1,ax2) = plt.subplots(1,2)
img1 = ax1.imshow(local_kernel,interpolation='none')
ax1.set_title(f'Width: {local_kernel.shape[0]*40}m')
img2 = ax2.imshow(regional_kernel,interpolation='none')
ax2.set_title(f'Width: {regional_kernel.shape[0]*40}m')

In [ ]:
%%time
params = [2,25]
outname = f'inputs/ifgramStack_{params[0]}_{params[1]}.h5'
!spatial_filter.py inputs/ifgramStack.h5 unwrapPhase -o {outname} -f double_difference -p {params[0]} {params[1]}

## directory management

In [ ]:
pwd

In [ ]:
mkdir inputs

In [ ]:
cp -r /home/micromamba/raw/desc_upper/mintpy/inputs/ifgramStack_2_25.h5  ./inputs/ifgramStack_2_25.h5

In [ ]:
cp -r /home/micromamba/raw/desc_upper/mintpy/inputs/geometryGeo.h5  ./inputs/geometryGeo.h5

In [ ]:
# rm -rf inputs

## generate water mask

In [ ]:
!generate_mask.py ./inputs/geometryGeo.h5 waterMask -m 0.5 -o waterMask.h5
view.main('waterMask.h5 --noverbose'.split())

## reference_point

In [ ]:
view.main('inputs/geometryGeo.h5 height -m waterMask.h5 --noverbose'.split())

In [ ]:
!temporal_average.py inputs/ifgramStack_2_25.h5 -d coherence -o avgSpatialCoh.h5

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
view.main('avgSpatialCoh.h5 --noverbose --figsize 8 8'.split())

In [ ]:
!reference_point.py inputs/ifgramStack_2_25.h5 --reset
!reference_point.py inputs/ifgramStack_2_25.h5 -t ./hyp3.cfg -m waterMask.h5

## modify_network

This step has the largest effect on the overall quality of the analysis so **take your time here and remove garbage data!**
Remember the wise words:
> Garbage in, Garbage out.

In [ ]:
!spatial_average.py inputs/ifgramStack.h5 -d coherence -m waterMask.h5

m = pd.read_csv('coherenceSpatialAvg.txt',skiprows=5,header=None,sep='\t',
                names=['DATE12','Mean','Btemp/days','Bperp/m','Num'])

m['date1'],m['date2'] = zip(*m['DATE12'].str.split('_'))
m['date1'] = pd.to_datetime(m['date1'])
m['date2'] = pd.to_datetime(m['date2'])

dates = sorted(list(set(list(m.date1) + list(m.date1))))

matrix = pd.DataFrame(index=dates,columns=dates)
for i,row in m.iterrows():
    matrix.loc[row['date1'],row['date2']] = row['Mean']
    matrix.loc[row['date2'],row['date1']] = row['Mean']
    
avg_coh = pd.DataFrame({'date':matrix.index,'coherence':matrix.mean(axis=1)}).reset_index(drop=True)

smooth_coh = avg_coh.copy()
smooth_coh.index = pd.DatetimeIndex(smooth_coh.date)
smooth_coh = smooth_coh.sort_index()
smooth_coh = smooth_coh.rolling(datetime.timedelta(days=60),min_periods=1).mean()
smooth_coh = smooth_coh.reset_index()

f, ax = plt.subplots(1,1,figsize=(15,5))

ax.plot(avg_coh.date,avg_coh.coherence)
ax.plot(smooth_coh.date,smooth_coh.coherence,color='red')
ax.set_xlabel('Date')
ax.set_ylabel('Coherence')
plt.savefig('coh_timeseries.png')
# plt.xlim((datetime.datetime(2021,3,1),datetime.datetime(2021,4,1)))

Based on this graph, there is a clear seasonal trend in the correlation data. I've found that performing the SBAS inversion on each year separately (breaking the analysis at the low points) produces the best results.

In [ ]:
cp /home/micromamba/scripts/configs/desc_upper.cfg ./hyp3.cfg

In [ ]:
!modify_network.py inputs/ifgramStack_2_25.h5 --reset
!modify_network.py inputs/ifgramStack_2_25.h5 -t hyp3.cfg

In [ ]:
plot_network.main('inputs/ifgramStack_2_25.h5 -t hyp3.cfg --show-kept --figsize 15 5'.split())

## view ifgram stack

In [ ]:
d = readfile.read_template('hyp3.cfg')
start_date = datetime.datetime.strptime(d['mintpy.network.startDate'], '%Y%m%d')
end_date = datetime.datetime.strptime(d['mintpy.network.endDate'], '%Y%m%d')
refy, refx = d['mintpy.reference.yx'].split(', ')
min_coh = float(d['mintpy.network.minCoherence'])

used_num = [str(x) for x in m.loc[(m.date1 >= start_date) & (m.date2 <= end_date), 'Num']]

In [ ]:
view.main(f'inputs/ifgramStack_2_25.h5 --noverbose --ref-yx {refy} {refx} -n {" ".join(used_num)} --vlim -3 3'.split())

## invert_network
This can take a while so write it to a log if you need to.

In [ ]:
# !ifgram_inversion.py inputs/ifgramStack.h5 -t ./hyp3.cfg |& tee -i invert.log ; python ../JPL_internship/emailme.py
!ifgram_inversion.py inputs/ifgramStack_2_25.h5 -t ./hyp3.cfg -m waterMask.h5 -o timeseries_descupper_2021.h5 temporalCoherence_descupper_2021.h5 numInvIfgram_descupper_2021.h5

## check temporal ramp

In [ ]:
ts = timeseries(file='timeseries_descupper_2021.h5')

data = ts.read()
data[data == 0] = np.nan
# data[:,~coh_mask] = np.nan
dates = pd.to_datetime(ts.get_date_list())
means = np.nanmean(data,axis=(1, 2))
means[0] = 0
# datetimes = np.array([datetime.strptime(x,'%Y%m%d').toordinal() for x in dates])

datetimes = np.array([int(x.toordinal()) for x in dates])
slope, intercept, r, p, se = linregress(datetimes,means)
fit = slope*datetimes+intercept
print(f'slope: {round(slope*100,4)} cm\nr2:{round(r**2,2)}')

plt.scatter(datetimes,means)
plt.plot(datetimes,fit,color='red')
plt.ylim(-0.02,0.02)
plt.xlabel('Ordinal Date')
plt.ylabel('Avg Timeseries Value')

## velocity

In [ ]:
%%time
outname = 'velocity_descupper_2021.h5'
!timeseries2velocity.py timeseries_descupper_2021.h5 -o {outname} --bootstrap --bc 100

In [ ]:
view.main(f'{outname} --noverbose -m no --vlim -5 5'.split())
# view.main('temporalCoherence_descupper_2021.h5 --noverbose --colormap RdBu_truncate --cm-vlist 0.0 0.7 1.0'.split())

## create mask file

In [ ]:
!generate_mask.py temporalCoherence_descupper_2021.h5 -m 0.01 -o maskTempCoh_descupper.h5

In [ ]:
view.main(f'maskTempCoh_descupper.h5 --noverbose'.split())

In [ ]:
ls *h5

In [ ]:
cp *h5 /mnt/i/hyp3_downloads/desc_upper/mintpy

## complete!